## <center>LEARN HOW TO LOAD A PRETRAINED EFFICIENTNETV2 MODEL<br><br>AND FINETUNE IT!!</center>

### **IMPORTS**

In [ ]:
# Installs
print("\n... PIP/APT INSTALLS STARTING ...\n")
!pip install -q pandarallel
!pip install -q tensorflow_model_optimization
!pip install -q --upgrade tensorflow_datasets
!pip install -q neural-structured-learning
print("... PIP/APT INSTALLS COMPLETE ...\n")

print("\n... IMPORTS STARTING ...\n")
print("\n\tVERSION INFORMATION")
# Machine Learning and Data Science Imports
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
from glob import glob
import warnings
import requests
import imageio
import IPython
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import time
import gzip
import ast
import sys
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm; tqdm.pandas();
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2
    
print("\n\n... IMPORTS COMPLETE ...\n")

print("\n... EFFICIENTDET SETUP STARTING ...")

# SET LIBRARY DIRECTORY
LIB_DIR = "/kaggle/input/google-automl-efficientdetefficientnet-oct-2021"

# To give access to automl files
sys.path.insert(0, LIB_DIR)
sys.path.insert(0, os.path.join(LIB_DIR, "automl-master"))
sys.path.insert(0, os.path.join(LIB_DIR, "automl-master", "efficientnetv2"))
    
# EfficientDET Module Imports
import cflags
import datasets
import effnetv2_configs
import preprocessing
import effnetv2_model
import hparams
import utils
from main_tf2 import TrainableModel
print("... EFFICIENTDET SETUP COMPLETE ...\n")

### **EXPLORE THE CONFIG**

In [ ]:
# See EfficientNetV2 Base Config
for k,v in  hparams.base_config.items(): print(k,v)

### **LOAD A PRETRAINED MODEL AND INFER**

#### Download the Weights, Label File, and A Couple Test Images

In [ ]:
def download_weights(model_name):
    """ Download the model checkpoints """
    if not os.path.isdir(f"/kaggle/working/{model_name}"):
        !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/v2/{model_name}.tgz
        !tar -xvzf ./{model_name}.tgz
        !rm -rf ./{model_name}.tgz
    return f"/kaggle/working/{model_name}"

def download_label_map_file(return_map=True, output_txtfile_name="/kaggle/working/labels_map.txt"):
    """ Download label map and parse """
    if not os.path.isfile(output_txtfile_name):
        !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientnet/eval_data/labels_map.txt -O {output_txtfile_name}
    if return_map:
        return ast.literal_eval(open(output_txtfile_name, "r").read())

def download_test_images(test_1_path="/kaggle/working/test_1.jpg", test_2_path="/kaggle/working/test_2.jpg"):
    """ Download some test images and return paths """
    if not os.path.isfile(test_1_path):
        !wget https://upload.wikimedia.org/wikipedia/commons/f/fe/Giant_Panda_in_Beijing_Zoo_1.JPG -O {test_1_path}
    if not os.path.isfile(test_2_path):
        !wget https://cdn.britannica.com/22/206222-131-E921E1FB/Domestic-feline-tabby-cat.jpg -O {test_2_path}
    return test_1_path, test_2_path
        
    
# Define which model and get the required helpers
model_name = "efficientnetv2-b0"
model_wt_path = download_weights(model_name)
lbl_map = download_label_map_file(); _small_lbl_map = {k:v for k,v in lbl_map.items() if k<10}
img_path_1, img_path_2 = download_test_images()
print(f"\n... MODEL NAME: {model_name} ...")
print(f"... MODEL WEIGHT PATH: {model_wt_path} ...\n")

print(f"\n... TESTING IMAGE PATH 1: {img_path_1} ...")
print(f"... TESTING IMAGE PATH 2: {img_path_2} ...\n")

print(f"\n... LBL MAP (FIRST TEN LABELS):\n\n{_small_lbl_map} ...\n")

#### Build The Model and Infer

In [ ]:
def ev2_load_and_preprocess(img_path, size=224):
    """ Load and preprocess an image given a path """
    return preprocessing.preprocess_image(tf.io.read_file(img_path), size, is_training=False)

print("\n... DEFINE IMAGE SIZE AND LOAD DEMO IMAGES AND PLOT ...\n")
IMG_SIZE = (224,224,3)
test_img_1 = ev2_load_and_preprocess(img_path_1)
test_img_2 = ev2_load_and_preprocess(img_path_2)

plt.figure(figsize=(20,10))

plt.subplot(1,4,1)
plt.imshow(cv2.imread(img_path_1)[..., ::-1])
plt.title("Original Test Image 1", fontweight="bold")
plt.axis(False)

plt.subplot(1,4,2)
plt.imshow(test_img_1)
plt.title(f"Preprocessed Test Image 1 - (SHAPE={IMG_SIZE})", fontweight="bold")
plt.axis(False)

plt.subplot(1,4,3)
plt.imshow(cv2.imread(img_path_2)[..., ::-1])
plt.title("Original Test Image 2", fontweight="bold")
plt.axis(False)

plt.subplot(1,4,4)
plt.imshow(test_img_2)
plt.title(f"Preprocessed Test Image 2 - (SHAPE={IMG_SIZE})", fontweight="bold")
plt.axis(False)

plt.tight_layout()
plt.show()


# For some reason this is required or else you will get an AssertionError on weight loading
print("\n\n\n... CLEAR THE BACKEND KERAS SESSION ...\n")
tf.keras.backend.clear_session()

# Load the model
print("\n... LOAD THE MODEL ...\n")
ev2 = effnetv2_model.EffNetV2Model(model_name=model_name, name=model_name)

# Build the model with a dummy call (ensure batch size is inline with what you intend to use)
print("\n... BUILD THE MODEL ...\n")
ev2(tf.ones((1,*IMG_SIZE)), training=False)

# Get the latest checkpoint from path and load into the model
print("\n... LOAD PRETRAINED WEIGHTS INTO THE MODEL ...\n")
ckpt = tf.train.latest_checkpoint(model_wt_path)
ev2.load_weights(ckpt)


# perform inference and take the output layer (index to 0)
print("\n... PERFORM INFERENCE W/ THE MODEL ...\n")
preds_1 = ev2(tf.expand_dims(test_img_1, axis=0), training=False)
preds_2 = ev2(tf.expand_dims(test_img_2, axis=0), training=False)

# Convert raw predictions into understandable information
top_k=3
print("\n... CONVERT PREDICTIONS INTO UNDERSTANDABLE INFORMATION ...\n")
print("\t--> TEST IMAGE 1 - TOP 10 PREDICTIONS")
for x in [f"\t\t#{i+1} MOST CONFIDENT\n\t\t\t----> LABEL={x[1]}\n\t\t\t----> SCORE={x[2]:.4f}" for i, x in enumerate(tf.keras.applications.efficientnet.decode_predictions(preds_1.numpy(), top=top_k)[0])]: print(x)
print("\n\t--> TEST IMAGE 2 - TOP 10 PREDICTIONS")
for x in [f"\t\t#{i+1} MOST CONFIDENT\n\t\t\t----> LABEL={x[1]}\n\t\t\t----> SCORE={x[2]:.4f}" for i, x in enumerate(tf.keras.applications.efficientnet.decode_predictions(preds_2.numpy(), top=top_k)[0])]: print(x)

print("\n\n\n... PLOT THE TOP PREDICTION AS THE TITLE TO THE ORIGINAL IMAGE PLOT ...\n\n")
plt.figure(figsize=(20,10))

plt.subplot(1,2,1)
plt.imshow(cv2.imread(img_path_1)[..., ::-1])
plt.title(f"Original Test Image 1 - LABEL={lbl_map[tf.argmax(preds_1, axis=-1)[0].numpy()]}", fontweight="bold")
plt.axis(False)

plt.subplot(1,2,2)
plt.imshow(cv2.imread(img_path_2)[..., ::-1])
plt.title(f"Original Test Image 2 - LABEL={lbl_map[tf.argmax(preds_2, axis=-1)[0].numpy()]}", fontweight="bold")
plt.axis(False)

plt.tight_layout()
plt.show()

### **FINETUNE THE MODEL FOR A SPECIFIC TASK**

**We will use two tasks and two datasets**

1. Petfinder Pawpularity Score

#### Create A `tf.data.Dataset` Object For Training and Validation

In [ ]:
# Load the dataframe
train_df = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")

# Discard all columns except Id and Pawpularity and rename to id and pawpularity
train_df = train_df[["Id", "Pawpularity"]]
train_df.columns = ["id", "pawpularity"]
train_df["pawpularity"] = train_df["pawpularity"]-1 # zero index

# Create a column containing path information for each id
TRAIN_IMG_DIR = "/kaggle/input/petfinder-pawpularity-score/train"
train_df["img_path"] = train_df["id"].apply(lambda x: os.path.join(TRAIN_IMG_DIR, x+".jpg"))

# Show demo plot
plt.figure(figsize=(8,8))
plt.imshow(ev2_load_and_preprocess(train_df.iloc[0].img_path))
plt.axis(False)
plt.title(f"Demo Image With First Row and Load Fn - Image Size={IMG_SIZE}", fontweight="bold")
plt.show()

N_VAL = int(len(train_df)/10)
val_df = train_df[:N_VAL]
train_df = train_df[N_VAL:]
N_TRAIN = len(train_df)
CLASSES = train_df["pawpularity"].unique()
N_CLASSES = len(CLASSES)

BATCH_SIZE=8
train_x = tf.data.Dataset.from_tensor_slices(train_df.img_path.values)
train_x = train_x.map(lambda x: ev2_load_and_preprocess(x, IMG_SIZE[0]), num_parallel_calls=tf.data.AUTOTUNE)
train_y = tf.data.Dataset.from_tensor_slices(train_df.pawpularity.values.astype(np.uint8))
train_ds = tf.data.Dataset.zip((train_x,train_y)).shuffle(BATCH_SIZE*8).batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.AUTOTUNE)

val_x = tf.data.Dataset.from_tensor_slices(val_df.img_path.values)
val_x = val_x.map(lambda x: ev2_load_and_preprocess(x, IMG_SIZE[0]), num_parallel_calls=tf.data.AUTOTUNE)
val_y = tf.data.Dataset.from_tensor_slices(val_df.pawpularity.values.astype(np.uint8))
val_ds = tf.data.Dataset.zip((val_x,val_y)).shuffle(BATCH_SIZE*8).batch(BATCH_SIZE,drop_remainder=True).prefetch(tf.data.AUTOTUNE)

print(f"\n... TRAIN DATASET OBJECT: {train_ds} ...")
print(f"... VAL   DATASET OBJECT: {val_ds} ...\n")

#### Update The Ev2 Model To Allow For Finetuning on A New Task (Regression)


In [ ]:
def get_paw_model(bb_trainable=False, model_name=model_name, dropout=0.2, weights="imagenet21k-ft1k"):
    """"""
    tf.keras.backend.clear_session(); gc.collect();
    _inputs = tf.keras.layers.Input(shape=IMG_SIZE)
    bb = effnetv2_model.get_model(model_name, include_top=False, weights=weights)
    
    if not bb_trainable:
        bb.trainable=False
        
    x = bb(_inputs)
    x = tf.keras.layers.Dropout(dropout)(x)
    _outputs = tf.keras.layers.Dense(1, activation="relu")(x)
    
    _model = tf.keras.Model(inputs=_inputs, outputs=_outputs)
    print(_model.summary())
    return _model
    
model = get_paw_model()
model.compile(optimizer="adam", loss=tf.keras.losses.MeanSquaredError(), metrics=tf.keras.metrics.RootMeanSquaredError())

#### Finetune the Model


In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=5)